<a href="https://colab.research.google.com/github/rmontteiro/ML_IGTI_bootcamp/blob/master/Desafio_Mod3_Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd #biblioteca para manipulação de dados
import numpy as np #biblioteca para utilizacao de vetores e matrizes
import matplotlib.pyplot as plt #bibloteca para plotar graficos
import seaborn as sns
import datetime

In [14]:
#importando o nosso dataset
from google.colab import files
uploaded = files.upload()

Saving wine.csv to wine (1).csv


In [15]:
df = pd.read_csv('wine.csv')

In [17]:
df.head()

,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [18]:
print(df)

     class  Alcohol  Malic_acid  ...   Hue  OD280%2FOD315_of_diluted_wines  Proline
0        1    14.23        1.71  ...  1.04                            3.92     1065
1        1    13.20        1.78  ...  1.05                            3.40     1050
2        1    13.16        2.36  ...  1.03                            3.17     1185
3        1    14.37        1.95  ...  0.86                            3.45     1480
4        1    13.24        2.59  ...  1.04                            2.93      735
..     ...      ...         ...  ...   ...                             ...      ...
173      3    13.71        5.65  ...  0.64                            1.74      740
174      3    13.40        3.91  ...  0.70                            1.56      750
175      3    13.27        4.28  ...  0.59                            1.56      835
176      3    13.17        2.59  ...  0.60                            1.62      840
177      3    14.13        4.10  ...  0.61                            1.60  

In [19]:
# Verificando o numero de amostras (linhas) e features (colunas) do dataset. 
print('Amostras e Features:', df.shape)

Amostras e Features: (178, 14)


In [20]:
# Verificando quais são os tipos das features
df.columns

Index(['class', 'Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash',
       'Magnesium', 'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols',
       'Proanthocyanins', 'Color_intensity', 'Hue',
       'OD280%2FOD315_of_diluted_wines', 'Proline'],
      dtype='object')

In [21]:
#verificando o formato do dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   class                           178 non-null    int64  
 1   Alcohol                         178 non-null    float64
 2   Malic_acid                      178 non-null    float64
 3   Ash                             178 non-null    float64
 4   Alcalinity_of_ash               178 non-null    float64
 5   Magnesium                       178 non-null    int64  
 6   Total_phenols                   178 non-null    float64
 7   Flavanoids                      178 non-null    float64
 8   Nonflavanoid_phenols            178 non-null    float64
 9   Proanthocyanins                 178 non-null    float64
 10  Color_intensity                 178 non-null    float64
 11  Hue                             178 non-null    float64
 12  OD280%2FOD315_of_diluted_wines  178 

In [22]:
df['class'].value_counts()

2    71
1    59
3    48
Name: class, dtype: int64

In [23]:
#contando valores nulos 
df.isnull().sum()

class                             0
Alcohol                           0
Malic_acid                        0
Ash                               0
Alcalinity_of_ash                 0
Magnesium                         0
Total_phenols                     0
Flavanoids                        0
Nonflavanoid_phenols              0
Proanthocyanins                   0
Color_intensity                   0
Hue                               0
OD280%2FOD315_of_diluted_wines    0
Proline                           0
dtype: int64

In [54]:
#criando um dicionario de dados para o mapeamento
name_to_class = {
    1: 0,
    2: 1,
    3: 2

}

#substituindo os valores categóricos pelo mapeamento
df['class'] = df['class'].map(name_to_class)

#check
df.head(5)

,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,0,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,0,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,0,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [55]:
# Analisando o resumo da base
df.describe()

,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,0.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,0.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,0.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,1.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,2.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,2.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [56]:
# armazenando os labels em um array
labels = np.array(df['class'])

# salvando a ordem das features
feature_list = list(df.columns)

In [57]:
# removendo a coluna de labels do df original
df2 = df.drop('class', axis = 1)

# check
df2.columns

Index(['Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash', 'Magnesium',
       'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols',
       'Proanthocyanins', 'Color_intensity', 'Hue',
       'OD280%2FOD315_of_diluted_wines', 'Proline'],
      dtype='object')

In [58]:
# convertendo df para array
data = np.array(df2)

In [59]:
# importar train_test_split do scikitlearn 
from sklearn.model_selection import train_test_split

# aplicando a funcao train_test_split para separar os conjuntos de treino e 
# teste segundo uma porcentagem de separação definida. 
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = 0.25, random_state = 42)


In [60]:
# criando baseline
baseline_preds = np.random.choice([0,1,2], size = len(test_labels))

print(baseline_preds)

[0 1 0 1 0 1 0 2 0 0 1 0 2 2 1 1 1 0 0 1 0 2 0 2 0 0 1 1 2 2 2 2 2 2 2 0 0
 0 0 0 2 2 2 2 1]


In [61]:
# importar biblioteca para calculo de métricas
from sklearn import metrics  
from sklearn.metrics import cluster

# Avaliando o baseline!
# essas medidas são calculadas a partir da comparação com o valor real do nosso conjunto de teste
print('Coeficiente de Silhueta\n', metrics.silhouette_score(test_data, baseline_preds)) 
print('\nDavies-Bouldin Score\n', metrics.davies_bouldin_score(test_data, baseline_preds)) 

print('\nMatriz de Contingência\n', metrics.cluster.contingency_matrix(test_labels, baseline_preds)) 
print('\nMutual information\n', metrics.mutual_info_score(test_labels, baseline_preds))

Coeficiente de Silhueta
 -0.020170277252158718

Davies-Bouldin Score
 75.52164512480151

Matriz de Contingência
 [[4 5 6]
 [6 5 7]
 [8 1 3]]

Mutual information
 0.06137641838182577


In [74]:
# importar o modelo de KMeans
from sklearn.cluster import KMeans

clustering = KMeans(n_clusters = 7, random_state = 42)
 
# treinando o modelo no conjunto de dados de treino
clustering.fit(train_data);

In [75]:
# aplicando o modelo treinado para a previsão da temperatura 
#em todo o conjunto de teste
predictions1_labels = clustering.predict(test_data)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
p = pd.DataFrame({'Real': test_labels, 'Previsto': predictions1_labels})  
p.head(10)

,Real,Previsto
0,0,6
1,0,0
2,2,4
3,0,3
4,1,4
5,0,3
6,1,4
7,2,2
8,1,2
9,2,6


In [76]:
#avaliando o modelo

print('Coeficiente de Silhueta\n', metrics.silhouette_score(test_data, predictions1_labels)) 
print('\nDavies-Bouldin Score\n', metrics.davies_bouldin_score(test_data, predictions1_labels)) 

print('\nMatriz de Contingência\n', metrics.cluster.contingency_matrix(test_labels, predictions1_labels)) 
print('\nMutual information\n', metrics.mutual_info_score(test_labels, predictions1_labels))

Coeficiente de Silhueta
 0.57470784353386

Davies-Bouldin Score
 0.4121556636418505

Matriz de Contingência
 [[7 0 0 4 0 2 2]
 [0 8 6 0 4 0 0]
 [0 0 5 0 5 0 2]]

Mutual information
 0.7177587344152296


# Desempenho com 7 clusters
Coeficiente de Silhueta
 0.57470784353386

Davies-Bouldin Score
 0.4121556636418505

Matriz de Contingência
 [[7 0 0 4 0 2 2]
 [0 8 6 0 4 0 0]
 [0 0 5 0 5 0 2]]

Mutual information
 0.7177587344152296

#Desempenho com 4 clusters
Coeficiente de Silhueta
 0.5574577694095691

Davies-Bouldin Score
 0.5422186226154052

Matriz de Contingência
 [[ 6  0  0  9]
 [ 0  7 11  0]
 [ 0  8  3  1]]

Mutual information
 0.6209931213798954

#Desempenho com 3 Clusters
Coeficiente de Silhueta
 0.5519241838976592

Davies-Bouldin Score
 0.5415115100039725

Matriz de Contingência
 [[13  0  2]
 [ 0 12  6]
 [ 0  5  7]]

Mutual information
 0.526053317723497

#Desepenho com 2 Clusters
Coeficiente de Silhueta
 0.6420626105479827

Davies-Bouldin Score
 0.5009476567689258

Matriz de Contingência
 [[ 0 15]
 [18  0]
 [12  0]]

Mutual information
 0.6365141682948124